In [7]:
import pandas as pd
from pypeds import ipeds

In [29]:
data = pd.read_csv('./data.csv')
ipeds_matches = pd.read_csv('./ipeds_matches.csv')
ipeds_matches.columns = ['_', 'a_names', 'ipeds_names', 'distance']
name_matches = pd.read_csv('./name_matches.csv')

In [10]:
hd_df = pd.read_csv('./hd_df.csv')
ic_df = pd.read_csv('./ic_df.csv')

In [11]:
_all = ic_df.merge(hd_df, how='inner', on = "unitid")

In [33]:
_all.columns

Index(['Unnamed: 0_x', 'unitid', 'peo1istr', 'peo2istr', 'peo3istr',
       'peo4istr', 'peo5istr', 'peo6istr', 'cntlaffi', 'pubprime',
       ...
       'necta', 'countycd', 'countynm', 'cngdstcd', 'longitud', 'latitude',
       'dfrcgid', 'dfrcuscg', 'survey_year_y', 'fall_year_y'],
      dtype='object', length=266)

In [30]:
ipeds_matches

,_,a_names,ipeds_names,distance
0,0,North Park University,North Park University,0.0
1,1,Oral Roberts University,Oral Roberts University,0.0
2,2,Lipscomb University,Lipscomb University,0.0
3,3,Saginaw Valley State University,Saginaw Valley State University,0.0
4,4,Clemson University,Clemson University,0.0
...,...,...,...,...
497,497,Eastern Mennonite University,Eastern Mennonite University,0.0
498,498,Dallas Baptist University,Dallas Baptist University,0.0
499,499,Manchester University,Manchester University,0.0
500,500,George Washington University,George Washington University,0.0


In [50]:
'''data_and_nm = data.merge(name_matches, how='inner', left_on = 'College', right_on = 'd_names')

data_ipeds = data_and_nm.merge(ipeds_matches, how = 'inner', on = 'a_names' )

combined = data_ipeds.merge(_all, how= 'inner', left_on = "ipeds_names", right_on = "instnm" )'''

'data_and_nm = data.merge(name_matches, how=\'inner\', left_on = \'College\', right_on = \'d_names\')\n\ndata_ipeds = data_and_nm.merge(ipeds_matches, how = \'inner\', on = \'a_names\' )\n\ncombined = data_ipeds.merge(_all, how= \'inner\', left_on = "ipeds_names", right_on = "instnm" )'

In [52]:
act_data = _all[['acten25', 'acten75', 'instnm']]

w_act = act_data.merge(pd.read_csv('./name_matches.csv'), how='inner', left_on='instnm', right_on='a_names')

w_act['actmed'] = (w_act.acten25 + w_act.acten75)/2

combined = w_act.merge(pd.read_csv('./data.csv'), left_on = 'd_names', right_on = 'College').dropna()

In [53]:
combined['actmed'] = (combined.acten25 + combined.acten75)/2
cols = ['actmed',
 'Position',
 'Class', 
 'Conference', 
 'latitude', 
 'longitude', 
 'cluster'
]

In [95]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [64]:
X = pd.get_dummies(combined[cols])

In [65]:
le = LabelEncoder()

In [71]:
y = le.fit_transform(combined['College'])

In [72]:
lm = LogisticRegression(max_iter = 300)

In [73]:
lm.fit(X,y)

C:\Users\Ty Gelman\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Ty Gelman\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=300,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [81]:
p1 = lm.predict_proba(X)[0]

In [89]:
p1list = list(enumerate(p1))
p1list.sort(key= lambda x: x[1], reverse=True)
[(le.inverse_transform([i]), p) for i,p in p1list]

[(array(['Alabama-Huntsville'], dtype=object), 0.3350094563929545),
 (array(['Lee'], dtype=object), 0.1893370142314327),
 (array(['West Florida'], dtype=object), 0.17586037218880263),
 (array(['Montevallo'], dtype=object), 0.07221320851326947),
 (array(['Spring Hill'], dtype=object), 0.030042195321608577),
 (array(['Covenant'], dtype=object), 0.006378734691731663),
 (array(['South Dakota School of Mines'], dtype=object),
  0.0050212681644814135),
 (array(['North Greenville'], dtype=object), 0.004458337706219019),
 (array(['Washington and Lee'], dtype=object), 0.004409654516633345),
 (array(['Kenyon'], dtype=object), 0.004353052013332225),
 (array(['Furman'], dtype=object), 0.003631593095353046),
 (array(['Michigan State'], dtype=object), 0.0035967163337207266),
 (array(['Truman State'], dtype=object), 0.0031795429681690203),
 (array(['Boston University'], dtype=object), 0.0030690799432350033),
 (array(['Grinnell'], dtype=object), 0.0030012538518958253),
 (array(['Massachusetts Institut

In [109]:
import numpy as np
r = np.array([X[c].sample(1).values for c in X]).T

In [91]:
from sklearn.naive_bayes import GaussianNB

In [92]:
bayes = GaussianNB()

In [93]:
bayes.fit(X,y)

GaussianNB(priors=None, var_smoothing=1e-09)

In [112]:
sorted(bayes.predict_proba(r)[0]*100)

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0